# Importing libraries

In [ ]:
# pip install pandas==1.5.3

import pandas as pd
from ydata_profiling import ProfileReport
from collections import Counter
import numpy as np

from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from yellowbrick.classifier import ClassificationReport
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns

# This version is depricated, so i used yt-dlp instead
#import youtube_dl
import yt_dlp
import csv

# loading 
from IPython.display import display
# tables
from tabulate import tabulate
import itertools

# -------------------------------------------------------------

# Downloading Data

User input

In [ ]:
global playlistend
global videosPerChannel
global Query_Name

playlistend = 50
videosPerChannel = 50
Query_Name = "gamejam po polsku"

Get search lookup

In [ ]:
urlTest="https://www.youtube.com/results?search_query="+Query_Name.replace(' ', '+')

url_list = []

In [ ]:
ydl_opts1 = {
    'noabortonerror': True,
    'ignoreerrors': True,
    'skip_download': True,
    'no_warnings': True,
    'writesubtitles': False,
    'writeautomaticsub': False,
    'allsubtitles': False,
    'listformats': False,
    'forcejson': False,
    'quiet': True,
    'no_warnings': True,
    'verbose': True,
    'playlistend': playlistend
}

with yt_dlp.YoutubeDL(ydl_opts1) as ydl:
    info =ydl.extract_info(urlTest, download=False)
    for entry in info['entries']:
        try:
            url_list.append(entry['uploader_url'])
        except Exception as e:
            print(f"Error occurred: {e}")
            continue
display(url_list)

adding your youtube chanell to discover your views

In [ ]:
my_youtube_channel = "https://www.youtube.com/@MroczneGranie"
url_list.append(my_youtube_channel)

Removing Duplicates

In [ ]:
url_list = list(set(url_list))
display(url_list)

Get content of channels (for training set)

In [ ]:
def normalizeText(text):
    return text.lower().replace(' ', '_').replace(',', ' ')

In [ ]:
def FetchChannel():
    global videosPerChannel

    ydl_opts = {
        'noabortonerror': True,
        'ignoreerrors': True,
        'skip_download': True,
        'no_warnings': True,
        'writesubtitles': False,
        'writeautomaticsub': False,
        'allsubtitles': False,
        'listformats': False,
        'forcejson': False,
        'quiet': True,
        'no_warnings': True,
        'verbose': True,
        'playlistend': videosPerChannel
    }

    video_data = []

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            for url in url_list:
                info = ydl.extract_info(url+"/videos", download=False)
                if info is not None and hasattr(info, 'get'):
                    videos = info.get('entries')

                    for video in videos:
                        if video is not None and hasattr(info, 'get'):
                            video_info = {
                                'Channel': normalizeText("(uploader)"+video.get('uploader', '')),
                                'Title': video.get('title', '').lower(),
                                'Views': video.get('view_count', ''),
                                'Description': video.get('description', '').lower(),
                                'Tags': normalizeText(','.join(["(tag)" + tag for tag in video.get('tags', [])])),
                                'PhrasesTitle': "",
                                'PhrasesDescription': ""
                            }
                            #display(video_info)
                            video_data.append(video_info)
                        
                        backup = pd.DataFrame(video_data)
                        backup.to_csv('backup.csv', index=False)
            df = pd.DataFrame(video_data)  # Create a dataframe from the video data
            return df

        except Exception as e:
            print(f'Error processing URL: {url}')
            print(e)
            return pd.DataFrame()  # Return an empty dataframe if an error occurs

In [ ]:
ChannelArray = FetchChannel()
ChannelArray.to_csv('result.csv', index=False)

In [ ]:
display(ChannelArray)

# Or import already Fetched data

In [ ]:
ChannelArray = pd.read_csv('result.csv')

# Common Pharses

Tuning value

In [ ]:
global numberOfPharses
numberOfPharses = 200

Functions

In [ ]:
def find_most_common_phrases(texts, saveDestination="phrases", n=1000, max_phrase_length=4):
    long_text = ' '.join(texts)
    #long_text = long_text.lower().replace(".", "").replace(",", "").replace("!", "").replace("?", "")
    words = long_text.split()

    phrases = []
    for phrase_length in range(1, max_phrase_length + 1):
        phrases += [' '.join(words[i:i+phrase_length]) for i in range(len(words)-phrase_length+1)]
    
    phrases = [s for s in phrases if len(s) >= 4]

    phrase_counts = Counter(phrases)

    with open(saveDestination+".csv", "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(phrase_counts.most_common(n))

    return phrase_counts.most_common(n)


In [ ]:
def findPharses(array,columnSource,columnDestination,prefix):
    # Assuming ChannelArray is a DataFrame with a column named 'Title'
    global numberOfPharses

    titles = ChannelArray[columnSource].tolist()
    common_phrases = find_most_common_phrases(titles,prefix,numberOfPharses)

    if common_phrases:
        common_phrases = [phrase for phrase, count in common_phrases]  # Extract only the phrases
        
        for index, row in ChannelArray.iterrows():
            matching_elements = []
            for common_phrase in common_phrases:
                if common_phrase in row[columnSource]:
                    matching_elements.append(prefix+common_phrase)
            
            #display(matching_elements)
            ChannelArray.at[index, columnDestination] = normalizeText(','.join(matching_elements))
            
    else:
        # Handle the case when common_phrases is empty
        # For example, you can set the 'CommonPhrases' column to empty
        ChannelArray[columnDestination] = ""

findPharses(ChannelArray,'Title','PhrasesTitle',"(title)")
findPharses(ChannelArray,'Description','PhrasesDescription',"(description)")

ChannelArray.to_csv('result.csv', index=False, encoding='utf-8')


In [ ]:
display(ChannelArray)

Types

In [ ]:
ChannelArray['Channel'] = ChannelArray['Channel'].astype(str)
ChannelArray['Title'] = ChannelArray['Title'].astype(str)
ChannelArray['Views'] = ChannelArray['Views'].astype('int64')
ChannelArray['Description'] = ChannelArray['Description'].astype(str)
ChannelArray['Tags'] = ChannelArray['Tags'].astype(str)
ChannelArray['PhrasesTitle'] = ChannelArray['PhrasesTitle'].astype(str)
ChannelArray['PhrasesDescription'] = ChannelArray['PhrasesDescription'].astype(str)

In [ ]:
print(ChannelArray.dtypes)

Profiling Raport

In [ ]:
ChannelArray.describe()

# Predictors and Targets

FOR COLLECTING TAGS, DESCRIPTION, NAME AND CHANNEL

In [ ]:
# Merging columns 1 5 6 and 7

columns_to_merge = [0, 4, 5, 6]

FOR COLLECTING TAGS AND CHANNEL ONLY

In [ ]:
columns_to_merge = [0, 4]

FOR COLLECTING TAGS AND CHANNEL AND TITLE

In [ ]:
columns_to_merge = [0, 4, 5]

Applying columns

In [ ]:
X = ChannelArray.apply(lambda row: ' '.join([str(row[col]) for col in columns_to_merge]), axis=1)
X = pd.DataFrame(X)
X = X.rename(columns={0: 'keywords'})
display(X)


In [ ]:
# Targets are just 2 column
columns_to_merge = [2]
Y = ChannelArray.apply(lambda row: ' '.join([str(row[col]) for col in columns_to_merge]), axis=1)
Y = pd.DataFrame(Y)
Y = Y.rename(columns={0: 'views'})
Y['views'] = Y['views'].astype('int64')
display(Y)

In [ ]:
print(Y.dtypes)

Create keyword series list (list for columns in bool table)

In [ ]:
#keywords_series = pd.DataFrame(X['keywords'].str.split().explode().unique()).rename(columns={0: 'keywords'})
keywords_series = X['keywords'].str.split().explode().unique()
display(keywords_series)

Pivot Table

In [ ]:

# Creating an empty DataFrame to store the transformed data
X_transformed = pd.DataFrame(index=range(len(X)), columns=keywords_series)

# Iterating over each element in X
for row, keywords in X.iterrows():
    for column in keywords_series:
        if column in keywords['keywords']:
            X_transformed.loc[int(row), str(column)] = True

X_transformed = X_transformed.fillna(False)

# Displaying the transformed DataFrame
display(X_transformed)
X_transformed.to_csv('pivotTable.csv', index=False, encoding='utf-8')



## Tuning Parameter

Tuning value for tags (1 because channel not count for tags)

In [ ]:
DeleteColumnsWithOnly = 4 #True values
deleteRowsWithOnly = 2 # Keywords
OutlinerPrecise  = 2.0 # For Z-score outliner delete points

# Delete rows with empty tags

In [ ]:
ChannelArray = ChannelArray.dropna(subset=['Tags'])

# Delete columns with only one instance

In [ ]:
# Count the number of "True" values in each column
counts = X_transformed.sum()

# Get the column names that have less than two "True" values
columns_to_delete = counts[counts < DeleteColumnsWithOnly].index

# Delete the columns from the DataFrame
X_transformed = X_transformed.drop(columns=columns_to_delete)

display(X_transformed)

# Delete rows with no tags

In [ ]:
# Count the number of "True" values in each row
counts = X_transformed.sum(axis=1)

# Get the row indices that have less than two "True" values
rows_to_delete = counts[counts < deleteRowsWithOnly].index

# Delete the rows from the DataFrame
X_transformed = X_transformed.drop(rows_to_delete)
Y = Y.drop(rows_to_delete)

# Transpose the DataFrame to reverse rows with columns
#X_transformed = X_transformed.T

display(X_transformed)


Merge X_transfered and Y

In [ ]:
merged_XY = pd.concat([X_transformed, Y], axis=1)
display(merged_XY)

# Outlier Points

Tuning Parameter for Outliner Points

In [ ]:
z_scores = sc.stats.zscore(Y)
abs_z_scores = np.abs(z_scores)
filtered_z_scores = (abs_z_scores < OutlinerPrecise).all(axis=1) #kind of mask
NewMerged_XY = merged_XY[filtered_z_scores]
NewMerged_XY

fix

In [ ]:
#--------------------------------------------------------- TAKE PARTS OF

X_transformed = NewMerged_XY.iloc[:, :-1]
Y = pd.DataFrame(NewMerged_XY.iloc[:, -1])

#--------------------------------------------------------- REMOVE COLUMNS WITH ONLY

# Count the number of "True" values in each column
counts = X_transformed.sum()

# Get the column names that have less than two "True" values
columns_to_delete = counts[counts < DeleteColumnsWithOnly].index

# Delete the columns from the DataFrame
X_transformed = X_transformed.drop(columns=columns_to_delete)

#--------------------------------------------------------- REMOVE ROWS WITH ONLY

# Count the number of "True" values in each row
counts = X_transformed.sum(axis=1)

# Get the row indices that have less than two "True" values
rows_to_delete = counts[counts < deleteRowsWithOnly].index

# Delete the rows from the DataFrame
X_transformed = X_transformed.drop(rows_to_delete)
Y = Y.drop(rows_to_delete)

# Transpose the DataFrame to reverse rows with columns
#X_transformed = X_transformed.T

#--------------------------------------------------------- MERGE AGAIN

merged_XY = pd.concat([X_transformed, Y], axis=1)
display(merged_XY)


## To do. 

### Delete Keywords X with keywords connected with only one channel

# Corelations

In [ ]:
correlation_matrix = NewMerged_XY.corr()
target_column = 'views'
correlations = correlation_matrix[target_column].drop(target_column)
print(pd.DataFrame(correlations).sort_values(by=target_column, ascending=False).head(50))

# Machine Learning

rate model acc in LOOCV 

accept +- error in acc

In [ ]:
acceptErrorInAccuracy = 25

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

List of models

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
from sklearn.svm import SVC
model = SVC()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

## Accuracy LOOCV

Accuracy 

In [ ]:
NewMerged_XY.to_csv("TrainingData.csv")
Xval = NewMerged_XY.iloc[:, :-1]
yval = NewMerged_XY.iloc[:, -1].values.ravel()

# Initialize LOOCV
loo = LeaveOneOut()

# Counter for predictions within the acceptable error range
within_range_count = 0

# Perform LOOCV
for train_index, test_index in loo.split(Xval):
    print(str(test_index[0])+"/"+str(len(Xval)))
    X_train, X_test = Xval.values[train_index], Xval.values[test_index]
    y_train, y_test = yval[train_index], yval[test_index]

    # Train the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)[0]

    # Check if the prediction falls within the acceptable error range
    if abs(y_pred - y_test[0]) <= acceptErrorInAccuracy:
        within_range_count += 1

# Calculate the percentage of predictions within the acceptable error range
within_range_percentage = (within_range_count / len(yval)) * 100
display("Percentage of predictions within the acceptable error range:", within_range_percentage)



LogisticRegression()

Percentage of predictions within the acceptable error range: 19.832985386221296

acceptErrorInAccuracy = 25

5 MIN 38 SEC


~ Accuracy: 0.006920415224913495 IN EXACT VALUE

2 min 48 sec

# Coefficients (Thats whats we wanted)

for logic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(Xval, yval)

# Retrieve the coefficients from the trained model
coefficients = model.coef_[0]

# Create a DataFrame to store the variable names and coefficients
coefficients_df = pd.DataFrame({'Variable': Xval.columns, 'Coefficient': coefficients})

# ---------------------------- COUNTING APPERIANCES ---------------------------------------------------------------
# Count the number of occurrences of each variable in X
variable_counts = Xval.astype(bool).sum(axis=0).reset_index()
variable_counts.columns = ['Variable', 'Count']
# Merge the variable counts with the coefficients DataFrame
coefficients_df = pd.merge(coefficients_df, variable_counts, on='Variable')


# Sort the coefficients by magnitude
coefficients_df = coefficients_df.sort_values(by='Coefficient', ascending=False)

# Display the variables and their coefficients
display(coefficients_df.head(25)) 


for random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
model = RandomForestClassifier()

# Train the classifier
model.fit(Xval, yval)  # X is your feature matrix, y is the target vector

# Get feature importances
feature_importances = model.feature_importances_

feature_names = Xval.columns

# Get the indices of the top k features
k = 25  # Set the number of top features you want to retrieve
top_k_indices = np.argsort(feature_importances)[-k:]

# Get the names of the top k features
top_k_feature_names = [feature_names[i] for i in top_k_indices]

display(top_k_feature_names)

# Generate HTML raport

Sklearn raport

In [ ]:
report = ClassificationReport(model, classes=classes, support=True)
report.fit(X_train, y_train)
report.score(X_test, y_test)
report.show(outpath="classification_report.html")

Pandas profile

~ 2 min 4.9 sec for 213 columns

In [ ]:
report = ProfileReport(NewMerged_XY)
report.to_file("profile_report.html")

# SMOTE